In [1]:
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import VGG16,ResNet50,MobileNetV2
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix


In [2]:
# Define paths
data_dir = '/Users/samuelsetsofia/dev/projects/EyeDisease_Classifier_DeepLearning/dataset'
classes = ['cataract', 'glaucoma', 'diabetic_retinopathy', 'normal']

# Create train, val, and test folders
output_dir = '/Users/samuelsetsofia/dev/projects/EyeDisease_Classifier_DeepLearning/output_dir'
os.makedirs(output_dir, exist_ok=True)
for folder in ['train', 'val', 'test']:
    for cls in classes:
        os.makedirs(os.path.join(output_dir, folder, cls), exist_ok=True)

# Split files
for cls in classes:
    class_path = os.path.join(data_dir, cls)
    files = np.array(os.listdir(class_path))
    train_files, temp_files = train_test_split(files, test_size=0.3, random_state=42)
    val_files, test_files = train_test_split(temp_files, test_size=0.5, random_state=42)

    for file_list, folder in zip([train_files, val_files, test_files], ['train', 'val', 'test']):
        for file in file_list:
            shutil.copy(os.path.join(class_path, file), os.path.join(output_dir, folder, cls))

### Load and Preprocess Images

In [3]:


train_dir = '/Users/samuelsetsofia/dev/projects/EyeDisease_Classifier_DeepLearning/output_dir/train'
val_dir = '/Users/samuelsetsofia/dev/projects/EyeDisease_Classifier_DeepLearning/output_dir/val'
test_dir = '/Users/samuelsetsofia/dev/projects/EyeDisease_Classifier_DeepLearning/output_dir/test'

batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 2949 files belonging to 4 classes.
Found 633 files belonging to 4 classes.
Found 635 files belonging to 4 classes.


### Model 1: VGG16 (Transfer Learning)
A deep convolutional neural network architecture that is effective for image classification tasks.

In [6]:


base_model = VGG16(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model_vgg16 = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')  # 4 classes
])

model_vgg16.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history_vgg16 = model_vgg16.fit(train_ds, validation_data=val_ds, epochs=10)

Epoch 1/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 227s 2s/step - accuracy: 0.6353 - loss: 10.9061 - val_accuracy: 0.8294 - val_loss: 0.4860
Epoch 2/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 228s 2s/step - accuracy: 0.7711 - loss: 0.5885 - val_accuracy: 0.8167 - val_loss: 0.4130
Epoch 3/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 234s 3s/step - accuracy: 0.8038 - loss: 0.4793 - val_accuracy: 0.8784 - val_loss: 0.3610
Epoch 4/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 236s 3s/step - accuracy: 0.8677 - loss: 0.3763 - val_accuracy: 0.8815 - val_loss: 0.3678
Epoch 5/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 235s 3s/step - accuracy: 0.8462 - loss: 0.3879 - val_accuracy: 0.8657 - val_loss: 0.3858
Epoch 6/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 234s 3s/step - accuracy: 0.8441 - loss: 0.3793 - val_accuracy: 0.8878 - val_loss: 0.3520
Epoch 7/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 234s 3s/step - accuracy: 0.8810 - loss: 0.3004 - val_accuracy: 0.8973 - val_loss: 0.3011
Epoch 8/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 231s 2s/step - accuracy: 0.8808 - loss: 0.2872 - val_accuracy: 0.8768 - 

### Model 2: ResNet50 (Transfer Learning)
A residual network that enables deeper networks by addressing vanishing gradient issues.

In [4]:


base_model = ResNet50(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model_resnet50 = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])

model_resnet50.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history_resnet50 = model_resnet50.fit(train_ds, validation_data=val_ds, epochs=10)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 20s 0us/step
Epoch 1/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 76s 779ms/step - accuracy: 0.6271 - loss: 1.0456 - val_accuracy: 0.8325 - val_loss: 0.4258
Epoch 2/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 71s 762ms/step - accuracy: 0.8255 - loss: 0.4619 - val_accuracy: 0.8720 - val_loss: 0.3540
Epoch 3/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 70s 750ms/step - accuracy: 0.8426 - loss: 0.4047 - val_accuracy: 0.8799 - val_loss: 0.3236
Epoch 4/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 70s 754ms/step - accuracy: 0.8630 - loss: 0.3500 - val_accuracy: 0.8784 - val_loss: 0.3086
Epoch 5/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 70s 751ms/step - accuracy: 0.8781 - loss: 0.3156 - val_accuracy: 0.8910 - val_loss: 0.2867
Epoch 6/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 70s 753ms/step - accuracy: 0.9052 - loss: 0.2821 - val_accuracy: 0.8942 - val_loss: 0.2698
Epoch 7/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 70s 754ms/step - accuracy: 0.8766 - loss: 0.3074 - val_accuracy: 0.8942 - val_loss: 0.3022
Epoch 8/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 70s 754

### Model 3: MobileNetV2 (Transfer Learning)
A lightweight convolutional neural network suitable for mobile and embedded applications.

In [5]:


base_model = MobileNetV2(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model_mobilenetv2 = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])

model_mobilenetv2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history_mobilenetv2 = model_mobilenetv2.fit(train_ds, validation_data=val_ds, epochs=10)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 27s 263ms/step - accuracy: 0.6264 - loss: 0.9472 - val_accuracy: 0.7804 - val_loss: 0.5309
Epoch 2/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 23s 244ms/step - accuracy: 0.7693 - loss: 0.5688 - val_accuracy: 0.7741 - val_loss: 0.5362
Epoch 3/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 23s 245ms/step - accuracy: 0.7810 - loss: 0.5326 - val_accuracy: 0.8183 - val_loss: 0.4637
Epoch 4/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 24s 262ms/step - accuracy: 0.8072 - loss: 0.4833 - val_accuracy: 0.8357 - val_loss: 0.4295
Epoch 5/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 26s 278ms/step - accuracy: 0.8187 - loss: 0.4462 - val_accuracy: 0.8199 - val_loss: 0.4376
Epoch 6/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 24s 258ms/step - accuracy: 0.8340 - loss: 0.4389 - val_accuracy: 0.8294 - val_loss: 0.4152
Epoch 7/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 24s 255ms/step - accuracy: 0.8416 - loss: 0.3954 - val_accuracy: 0.8231 - val_loss: 0.4101
Epoch 8/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 25s 264ms/

### Evaluate Model Performance
Confusion Matrix: For detailed class-level performance.
Precision, Recall, F1-Score: For assessing balance between sensitivity and specificity.

In [7]:
vgg16_loss, vgg16_accuracy = model_vgg16.evaluate(test_ds)
resnet50_loss, resnet50_accuracy = model_resnet50.evaluate(test_ds)
mobilenetv2_loss, mobilenetv2_accuracy = model_mobilenetv2.evaluate(test_ds)


true_labels = np.concatenate([y for x, y in test_ds], axis=0)

vgg16_predictions = np.argmax(model_vgg16.predict(test_ds), axis=1)
resnet50_predictions = np.argmax(model_resnet50.predict(test_ds), axis=1)
mobilenetv2_predictions = np.argmax(model_mobilenetv2.predict(test_ds), axis=1)

# Print classification reports
print("VGG16:")
print(classification_report(true_labels, vgg16_predictions))
print("ResNet50:")
print(classification_report(true_labels, resnet50_predictions))
print("MobileNetV2:")
print(classification_report(true_labels, mobilenetv2_predictions))

20/20 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.8857 - loss: 0.3236
20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 618ms/step - accuracy: 0.9096 - loss: 0.2426
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 210ms/step - accuracy: 0.8243 - loss: 0.3829


2024-12-17 01:54:56.449963: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


20/20 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 14s 660ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 233ms/step
VGG16:
              precision    recall  f1-score   support

           0       0.22      0.21      0.22       156
           1       0.21      0.21      0.21       165
           2       0.20      0.22      0.21       152
           3       0.22      0.20      0.21       162

    accuracy                           0.21       635
   macro avg       0.21      0.21      0.21       635
weighted avg       0.21      0.21      0.21       635

ResNet50:
              precision    recall  f1-score   support

           0       0.23      0.23      0.23       156
           1       0.18      0.19      0.19       165
           2       0.23      0.22      0.23       152
           3       0.24      0.24      0.24       162

    accuracy                           0.22       635
   macro avg       0.22      0.22      0.22       635
weighted avg       0.22      0.22      0.22   